In [57]:
import pandas as pd

In [58]:
data_path = "C:/Users/Katta's/PycharmProjects/data/usecase3/train.csv"
housing = pd.read_csv(data_path)

In [59]:
 house_obj= housing.select_dtypes(include=['object']).copy()

In [60]:
house_obj.drop(house_obj.columns[house_obj.isnull().any()],axis=1, inplace=True) 

In [61]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in house_obj.columns.values:
        le.fit(house_obj[col])
        house_obj[col]=le.transform(house_obj[col])

In [62]:
# All types except object
housing_int = housing.select_dtypes(exclude=['object']).copy()
# housing_cat1 = housing_cat.astype('int64',copy=False)
int_columns = housing_int.columns


In [63]:
from sklearn.preprocessing import StandardScaler
# Scaling training because of variance
ss = StandardScaler()
# scaling training data
housing_cat1_scaled = ss.fit_transform(housing_int)


c:\users\katta's\venv\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\katta's\venv\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [64]:
housing_cat1_scaled_df = pd.DataFrame(housing_cat1_scaled, columns=int_columns)

In [65]:
housing_t = pd.concat([house_obj, housing_cat1_scaled_df],axis= 1)


In [66]:
housing_t['GarageYrBlt'].fillna(housing_t['GarageYrBlt'].mean(), inplace=True)
housing_t['MasVnrArea'].fillna(housing_t['MasVnrArea'].mean(), inplace=True)
housing_t['LotFrontage'].fillna(housing_t['LotFrontage'].mean(), inplace=True)

In [67]:
housing_train = housing_t.dropna()
print(housing_train.columns)

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive',
       'SaleType', 'SaleCondition', 'Id', 'MSSubClass', 'LotFrontage',
       'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')


In [68]:
train_x = housing_train.drop(['SalePrice', 'Id'],axis=1)
train_y = housing_train['SalePrice']
train_x.columns

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive',
       'SaleType', 'SaleCondition', 'MSSubClass', 'LotFrontage', 'LotArea',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

In [69]:
# Early stopping 
from keras.callbacks import EarlyStopping
_es_monitor = 'val_loss'
_es_patience = 10
es = EarlyStopping(monitor=_es_monitor, mode='min', verbose=1, patience=_es_patience)

In [70]:
#define batch size and number of epchos 
_batch_size = 32
_epochs = 500
_mc_model_location = 'v1_model.h5'
_mc_monitor = 'val_loss'
lr = 0.005
# Defining optimizer 
from keras.optimizers import SGD
_opt = SGD(lr)
_loss = 'mean_squared_error'

In [82]:
# Model check point
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Dense(64, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='relu'))
mc = ModelCheckpoint(_mc_model_location, monitor=_mc_monitor, mode='max', verbose=1, save_best_only=True)

In [83]:
model.compile(loss=_loss, optimizer=_opt, metrics=['accuracy'])

In [88]:
history = model.fit(train_x.values, train_y.values, validation_split=0.20,
                    epochs=_epochs, batch_size = len(train_x), verbose=1, callbacks=[mc])

Train on 1168 samples, validate on 292 samples
Epoch 1/500
1168/1168 [==============================] - 0s 28us/step - loss: 0.9846 - acc: 0.0000e+00 - val_loss: 1.0635 - val_acc: 0.0000e+00



Epoch 00001: val_loss did not improve from 1.06348
Epoch 2/500
1168/1168 [==============================] - 0s 27us/step - loss: 0.9915 - acc: 0.0000e+00 - val_loss: 1.0635 - val_acc: 0.0000e+00



Epoch 00002: val_loss did not improve from 1.06348
Epoch 3/500
1168/1168 [==============================] - 0s 24us/step - loss: 0.9817 - acc: 0.0000e+00 - val_loss: 1.0635 - val_acc: 0.0000e+00



Epoch 00003: val_loss did not improve from 1.06348
Epoch 4/500
1168/1168 [==============================] - 0s 10us/step - loss: 0.9873 - acc: 0.0000e+00 - val_loss: 1.0635 - val_acc: 0.0000e+00



Epoch 00004: val_loss did not improve from 1.06348
Epoch 5/500
1168/1168 [==============================] - 0s 9us/step - loss: 0.9876 - acc: 0.0000e+00 - val_loss: 1.0635 - val_acc: 0.0000e+00


...


Epoch 00498: val_loss did not improve from 1.06348
Epoch 499/500


1168/1168 [==============================] - 0s 6us/step - loss: 0.9885 - acc: 0.0000e+00 - val_loss: 1.0635 - val_acc: 0.0000e+00



Epoch 00499: val_loss did not improve from 1.06348
Epoch 500/500


1168/1168 [==============================] - 0s 8us/step - loss: 0.9841 - acc: 0.0000e+00 - val_loss: 1.0635 - val_acc: 0.0000e+00



Epoch 00500: val_loss did not improve from 1.06348


In [1]:
# plot loss - Train vs Valid
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='test loss')
plt.title('Loss - Train vs validation')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [87]:
metrics = model.evaluate(train_x, train_y)
print("\n%s: %.2f%%" % (model.metrics_names[0], metrics[0]*100))

  32/1460 [..............................] - ETA: 0s

1460/1460 [==============================] - 0s 27us/step



loss: 100.00%
